# 05. CODE GENERATION

1. [x] введение
2. [x] [Codex](https://openai.com/blog/openai-codex/)
3. [x] [AlphaCode](https://www.deepmind.com/blog/competitive-programming-with-alphacode)
5. [x] упражение
6. [x] ссылки

# 1. Введение

| Model       | Company      | Date |  Model |
|-------------| -------------| -----|-----|
| [GPT-J](https://6b.eleuther.ai/)       | Eleuther.ai  | 2021 | https://github.com/kingoflolz/mesh-transformer-jax/#gpt-j-6b |
| [GPT-Neo](https://huggingface.co/EleutherAI/gpt-neo-2.7B)    | Eleuther.ai  | 2021 | https://github.com/EleutherAI/gpt-neo |
| [GPT-NeoX](https://huggingface.co/docs/transformers/model_doc/gpt_neox)| Eleuther.ai  | 2021 | https://github.com/EleutherAI/gpt-neox |
| [Codex](https://openai.com/blog/openai-codex/)       | OpenAI | 2021 | - |
| [CodeT5](https://blog.salesforceairesearch.com/codet5/)      | Salesforce   | 2021 | https://github.com/salesforce/CodeT5 |
| [CodeParrot](https://github.com/huggingface/transformers/tree/master/examples/research_projects/codeparrot)  | Hugging Face | 2021 | https://huggingface.co/lvwerra/codeparrot |
| [AlphaCode](https://www.deepmind.com/blog/competitive-programming-with-alphacode)   | Deepmind  | 2022 | - |
| [InCoder](https://sites.google.com/view/incoder-code-models) | Meta | 2022 | https://github.com/dpfried/incoder/blob/main/README.md |
| [CodeWhisperer](https://aws.amazon.com/codewhisperer/) | Amazon | 2022 | - |
| [CodeGen](https://arxiv.org/abs/2203.13474) | Salesforce | 2022 | https://github.com/salesforce/CodeGen |
| [PanGu-Coder](https://arxiv.org/abs/207.11280) | Huawei | 2022 | - |
| [PaLM-Coder](https://arxiv.org/abs/2204.02311) | Google | 2022 | - |

## CodeParrot

In [ ]:
# CodeParrot
from transformers import pipeline

pipe = pipeline('text-generation', model='lvwerra/codeparrot')
#pipe('def hello_world():')

In [ ]:
text = 'def add_numbers(a, b):\n    """add two numbers"""'

code = pipe(text)[0]['generated_text']

print(code)

## InCoder

- https://huggingface.co/spaces/facebook/incoder-demo
- https://huggingface.co/facebook/incoder-6B
- https://github.com/dpfried/incoder
- https://github.com/dpfried/incoder/blob/main/example_usage.py

# 2. Codex


![](./res/05_codex_paper.png)

Codex ([OpenAI](https://openai.com/), 2021) --- это языковая модель [GPT](https://openai.com/blog/language-unsupervised/), зафайнтюненная (fine-tuned) на общедоступном коде из GitHub (Python). На одной из версий этой модели построен [GitHub Copilot](https://copilot.github.com/) (Visual Studio Code, Visual Studio, Neovim и JetBrains IDE).

![](https://nira.com/wp-content/uploads/2021/11/image4-1-620x308.png)

Кроме модели Codex в той же статье был предложен датасет [HumanEval](https://github.com/openai/human-eval) --- составленный вручную датасет для оценки моделей генерации кода.

Поскольку входом для модели генерации кода служит текст (описание задачи) на естественном языке, то кажется естественным строить модель на основе файнтюнинга (fine-tuning) семейства моделей [GPT-3](https://arxiv.org/abs/2005.14165), которые уже имеют представление об естественном языке. Кстати, модель GPT-3 до файнтюнинга уже могла генерировать простые программы на языке Python из docstring-ов.

Из-за большого размера данных для fine-tuning-а выигрыша в качестве от дообучения предобученной модели нет. Но поскольку предобученная модель сходится быстрее, использовались предобученные модели.

## Данные для обучения

1. 54 миллионов общедоступных репозиториев с GitHub-а
2. 179 ГБ уникальных файлов Python размером менее 1 МБ
3. отфильтрованы файлы, которые имели среднюю длину строки более 100, максимальную длину строки более 1000 или содержали небольшой процент буквенно-цифровых символов (вероятно, были созданы автоматически)
4. окончательный набор данных --- 159 ГБ

## Обучение

Обучение состоит в файнтюнинге предобученной модели GPT-3.
![](https://jalammar.github.io/images/gpt3/04-gpt3-generate-tokens-output.gif)

- скорость обучения (learning rate) --- такая же, как при обучении соответствующей GPT
- 175 шагов линейного разогрева (warmup, линейный рост скорости обучения в начале)
- косинусное затуханием скорости обучения
- 100 миллиардов токенов
- оптимизатор Adam с $\beta_1 = 0.9$, $\beta_2 = 0.95$, $\epsilon = 10^{-8}$
- коэффициент снижения (weight decay) веса $0.1$ (добавяем слагаемое с коэффициентом в функцию потерь и поощряем меньше веса: $L_{new}(w) = L_{original}(w) + \lambda w^Tw$).

Для того, чтобы максимально использовать текстовые представления из GPT, авторы делали лексический анализ кода на основе токенизатора GPT-3 (BPE).
Поскольку распределение слов в коде GitHub отличается от распределения в обычном тексте, этот токенизатор не очень эффективен для код.
Самый большой источник неэффективности возникает из-за кодирования пробелов, поэтому авторы добавили дополнительный набор токенов для представления пробелов разной длины.
Это позволило сократить количество токенов примерно на 30%.

## Тестирование (evaluation)

Для тестирования был разработан набор задач по программированию HumanEval.
- 164 задачи
- задача: сигнатура функции, строка документации, тело и несколько тестов (в среднем, 7.7 тестов на задачу)
- задачи написаны заново, чтобы не было пересечения с GitHub: например, существует более десяти общедоступных репозиториев, содержащих решения задач [Codeforces](https://codeforces.com/)
- задачи программирования в наборе данных HumanEval оценивают понимание языка, рассуждения, алгоритмы и простую математику
- доступны: https://github.com/openai/human-eval

Оценивалась функциональная правильность с помощью метрики `pass@k`:
> для каждой задачи генерируется $k$ образцов кода, задача считается решенной, если какой-либо образец проходит тесты, и указывается общая доля решенных проблем.

Чтобы вычислить `pass@k', преобразуем каждую задачу HumanEval в prompt, состоящий из заголовка, сигнатуры и строки документации.

Prompt, отправляемый в  модель, отображается на белом фоне, а успешный код, созданный моделью, отображается на желтом фоне.

![Prompt](https://production-media.paperswithcode.com/datasets/a2af6cf1-212b-4a05-8d5c-55170a21ce05.png)

Конец генерации --- берем токены из Codex, пока не встретим одну из следующих стоп-последовательностей:
- `\nclass`
- `\ndef`
- `\n#`
- `\nif` или
- `\nprint`,

поскольку в противном случае модель продолжаит генерировать дополнительные функции или операторы.
Использовалось *nucleus sampling* со значением $p = 0.95$ (*top p sampling*, берём все варианты, которые в сумме дают вероятность $p$).

![Evaluation](./res/05_codex_table.png)

# 3. AlphaCode


![](./res/05_alphacode_paper.png)

Несмотря на достижения в области решении задач с помощью машинного обучения, подходы ограничивались относительно простыми задачами по математике и программированию, или поиском и копированием существующих решений.

Одна из задач AlphaCode --- создавать компьютерные программы на конкурентоспособном уровне. Во время тестирования AlphaCode вошла в число 54% лучших участников соревнований по программированию, решая новые задачи, требующие сочетания критического мышления, логики, алгоритмов, кодирования и понимания естественного языка.

## Пример

![Example](https://i0.wp.com/bdtechtalks.com/wp-content/uploads/2022/02/DeepMind-AlphaCode-visualization.png?ssl=1)


Ещё примеры:
- https://alphacode.deepmind.com/

## Данные

### Pre-training dataset

- срез некоторых публичных репозиториев GitHub, взятых на 2021-07-14
- языки программирования: C++, C#, Go, Java, JavaScript, Lua, PHP, Python, Ruby, Rust, Scala и TypeScript
- исключены все файлы размером более 1 МБ или со строками длиннее 1000 символов (автогенерация)
- исключены дубликаты, игнорируя пробелы при сравнении

Получилось 715 ГБ кода

### CodeContests fine-tuning dataset

Был создан датасет CodeContests:
- задачи
- решения
- тесты

Данные были извлечены с платформы [Codeforces](https://codeforces.com/) + [Description2Code](https://github.com/ethancaballero/description2code) + [CodeNet](https://github.com/IBM/Project_CodeNet).

Validation- и test-части --- это новые задачи из Codeforces. Разделение на основе времени.

![Dataset](./res/05_alphacode_dataset.png)

## Подход

![Overview](./res/05_alphacode_overview.png)

1. Предобучаем языковую модель на основе транформеров, используя код GitHub
2. Файнтюним модель на данных с соревнований
3. При решении каждой задачи генерируется большое количество решений
4. Фильтрация решений. Получаем небольшой набор решений, которые отправляем на проверку

## Модель

- трансформер, энкодер-декодерная архитектура
- асимметричная архитектура с 1536 токенов для энкодера и 768 токенов для декодера: описание задачи в среднем в два раза длиннее человеческого решения
- неглубокий энкодер и глубокий декодер
- [SentencePiece](https://github.com/google/sentencepiece) в качестве токенизатора (общий для энкодера и декодера), словарь содержал 8000 токенов

![Parameters](./res/05_alphacode_params.png)

## Обучение

### Pre-training

- GitHub dataset
- standard cross-entropy next-token prediction loss for the decoder
- masked language modeling loss for the encoder
- base 1B parameter model was trained for 106 steps with a batch size of 256
- AdamW (Loshchilov and Hutter, 2017), $\beta_1 = 0.9$, $\beta_2 = 0.999$ for {300M, 1B, 3B} models, and $\beta_2 = 0.95$ for {9B, 41B} models
- weight decay of 0.1 to enhance regularization
- initial learning rate of $10^{−4}$, which was then cosine decayed to $10^{-5}$ at the end of pre-training
- linearly warmed-up the learning rate from $10^{−9}$ to $10^{−4}$ over the first 1000 training steps
- clipped the global gradient norm to stay below 1.0

### Fine-tuning

- CodeContests dataset
- natural language problem description for the encoder and the program solution for the decoder
- used both the standard next-token prediction and masked language modeling losses
- initial learning rate as $10^{−5}$, and cosine decayed it to $10^{−6}$ at the end of fine-tuning
- used the same linear warm-up stage for the learning rate over the first 1000 training steps

ALSO:

- Tempering
- *Value conditioning & prediction*. CodeContests contains both correct and incorrect problem submissions. We used value conditioning and prediction to discriminate between these two types of submissions, providing an additional training signal and allowing use of data which could otherwise mislead the model. Similar approaches were used in, e.g., Vinyals et al. (2019). In value conditioning, we inserted whether or not a submission was correct into the problem description so that the model can condition on this information, as shown in Figure 5. At sampling time, the model was always conditioned on the sample being correct. In value prediction, we added an auxiliary value prediction task during training such that the last layer token representations before projecting to logits are also used in a small Transformer to classify whether the submission is correct. Value prediction was not used during sampling
- *GOLD* (Pang and He, 2020). Solving competitive programming problems from descriptions is inherently a one-of-many task (Nandwani et al., 2021): each unique problem allows many distinct solutions that depend on algorithm choice, implementation, etc. CodeContests contains several orders of magnitude more solutions than descriptions (Table 1). Standard maximum likelihood objectives minimise loss by putting some weight on each solution in the training set (like recall), whereas our metric measures whether a model can find a single correct solution in the submission attempt budget (like precision). To resolve this discrepancy, we adopted a variation of GOLD (Pang and He, 2020), an offline RL algorithm which allows the model to both learn from tokens it already assigns high likelihood to, and to ignore tokens that are not in its distribution (allowing it to concentrate on precision). To combine GOLD and tempering, we introduce a short training phase between pretraining and finetuning

## Генерация

### Large scale sampling

- генерировать половину решений на Python и половину на C++
- случайно накидывать тэги и уровни сложности для задач в качестве natural language prompt
- использовать относительно высокую температуру при генерации

В итоге имеем миллионы решения для одной задачи


### Filtering

Обычно на соревнованиях количество попыток ограничено. Здесь ограничение -- 10 попыток. Поэтому используются тесты из задачи для фильтрации. Так фильтрируется около 99% решений. Примерно для 10% задая не находится решений, которые бы прошли тесты.

### Clustering

- генерируются новые тестовые данные
- решения объединяются в кластеры по ответам на этих тестовых данных
- из каждого кластера, начиная с самого большого, выбираются представители

Для генерации тестовых данных используется отдельная небольшая специально обученная модель

### Результаты

![Results](./res/05_alphacode_results.png)

# 4. Упражнение

Провести эксперимент с генерацей кода с помощью модели InCoder. Исследовать влияние описания задачи, наличия юнит-тестов и т.д. на качество сгенерированного кода.

# 5. Полезные ссылки

- https://github.com/saltudelft/ml4se#code-generation
- [GPT-3](https://jalammar.github.io/how-gpt3-works-visualizations-animations/)